In [46]:
import cv2
import numpy as np
  


In [47]:
# Open the image files.
img1_color = cv2.imread("with2.jpg")  # Image to be aligned.
img2_color = cv2.imread("without.jpeg")    # Reference image.
  
# Convert to grayscale.
img1 = cv2.cvtColor(img1_color, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(img2_color, cv2.COLOR_BGR2GRAY)
height, width = img2.shape



In [48]:
# Create ORB detector with 5000 features.
orb_detector = cv2.ORB_create(5000)
  
# Find keypoints and descriptors.
# The first arg is the image, second arg is the mask
#  (which is not required in this case).
kp1, d1 = orb_detector.detectAndCompute(img1, None)
kp2, d2 = orb_detector.detectAndCompute(img2, None)
  
# Match features between the two images.
# We create a Brute Force matcher with 
# Hamming distance as measurement mode.
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True)

# Match the two sets of descriptors.
matches = list(matcher.match(d1, d2))
  
print(len(matches))

# Sort matches on the basis of their Hamming distance.
matches.sort(key = lambda x: x.distance)
  
# Take the top 90 % matches forward.
matches = matches[:int(len(matches)*0.93)]
no_of_matches = len(matches)
  


416


In [49]:
# Define empty matrices of shape no_of_matches * 2.
p1 = np.zeros((no_of_matches, 2))
p2 = np.zeros((no_of_matches, 2))
  
for i in range(len(matches)):
  p1[i, :] = kp1[matches[i].queryIdx].pt
  p2[i, :] = kp2[matches[i].trainIdx].pt
  
# Find the homography matrix.
homography, mask = cv2.findHomography(p1, p2, cv2.RANSAC)
  


In [50]:
# Use this matrix to transform the
# colored image wrt the reference image.
transformed_img = cv2.warpPerspective(img1_color,
                    homography, (width, height))

In [51]:
cv2.imwrite('with_cmp.jpg', transformed_img)

True

In [52]:

gray_img = cv2.cvtColor(transformed_img, cv2.COLOR_BGR2GRAY)
# Save the output.


In [53]:
mask = (gray_img > 0).astype(np.uint8) * 255
print(mask)

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]


In [54]:
masked_orig = cv2.bitwise_and(img2_color, img2_color, mask=mask)


In [55]:
cv2.imwrite('without_cmp.jpg', masked_orig)

True

In [56]:
print(len(matches))

386
